In [ ]:
# https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/
# 
# # https://archive.ics.uci.edu/ml/datasets/Air+Quality
# https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip

In [51]:
#!pip install wget

In [61]:
file = r'D:\github\1-Projects\0Experiments\AirQualityUCI\AirQualityUCI.csv'
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#read the data
df = pd.read_csv(file,sep=";",parse_dates=[['Date', 'Time']])

#check the dtypes
df.dtypes



Date_Time         object
CO(GT)           float64
PT08.S1(CO)        int64
NMHC(GT)           int64
C6H6(GT)         float64
PT08.S2(NMHC)      int64
NOx(GT)            int64
PT08.S3(NOx)       int64
NO2(GT)            int64
PT08.S4(NO2)       int64
PT08.S5(O3)        int64
T                float64
RH               float64
AH               float64
Unnamed: 15      float64
Unnamed: 16      float64
dtype: object

In [62]:
df=df.drop([ 'Unnamed: 15','Unnamed: 16'] , axis=1)

In [63]:
df['Date_Time'] = pd.to_datetime(df.Date_Time , format = '%d/%m/%Y %H.%M.%S')
data = df.drop(['Date_Time'], axis=1)
data.index = df.Date_Time

In [84]:
df

,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10 18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,2004-03-10 19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,2004-03-10 20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,2004-03-10 21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,2004-03-10 22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,2005-04-04 10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,2005-04-04 11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,2005-04-04 12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,2005-04-04 13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


In [64]:
#missing value treatment
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
       if data[j][i] == -200:
           data[j][i] = data[j][i-1]

#checking stationarity
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues

johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
coint_johansen(johan_test_temp,-1,1).eig

C:\Users\admin\AppData\Local\Temp/ipykernel_3296/2272647166.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[j][i] = data[j][i-1]


array([1.75510896e-01, 1.52389933e-01, 1.15120416e-01, 1.04126281e-01,
       9.29485509e-02, 6.89397159e-02, 5.77070988e-02, 3.43554214e-02,
       3.05980659e-02, 1.18697142e-02, 2.46766099e-03, 7.09584856e-05])

In [66]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
d:\Users\admin\anaconda3\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [91]:
model_fit.coefs.shape

(1, 13, 13)

In [96]:
pd.DataFrame(model_fit.coefs[0], columns=valid.columns) 
# betas learned of the model. It must be square matrix. Because there are 13 columns therefore it is 13x13 matrix. Rows have no specific meaning

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,0.760587,0.000777,-0.000398,-0.044539,0.001638,9.058883e-04,9.606349e-06,-0.001297,0.000077,-0.000523,0.003626,0.001731,-0.192640
1,-15.733829,1.018804,-0.053305,-6.255338,0.143308,1.252939e-01,5.284435e-03,-0.251679,0.014633,-0.037232,1.852306,0.537821,-45.723241
2,-2.850224,0.020808,0.827579,-0.781889,0.033226,1.979890e-02,-7.242704e-03,-0.026618,-0.001632,-0.017345,0.027981,-0.007577,-0.711648
3,-0.714694,0.004021,-0.001301,0.377372,0.013735,6.495028e-03,7.640048e-04,-0.011710,0.000847,-0.001808,0.064257,0.015353,-1.566420
4,-23.356748,0.121772,-0.034525,-11.548439,1.184533,2.106676e-01,1.732796e-02,-0.363672,0.041255,-0.054370,2.611170,0.617505,-62.302180
5,-8.001473,0.040911,-0.008028,-0.467098,0.122874,9.426144e-01,-1.965447e-02,-0.297356,-0.057169,-0.037929,0.709682,0.641582,-20.289814
6,15.898287,-0.100754,0.005658,-0.018139,0.035404,-1.277016e-01,9.155979e-01,0.141516,-0.001800,0.029331,-1.947006,-0.428880,25.743511
7,-1.580605,0.016921,0.000349,-0.013325,0.011901,1.778719e-02,-8.012378e-03,0.822734,-0.001524,-0.012549,0.080106,0.020290,-10.074910
8,-21.100224,0.129392,-0.070466,-16.761087,0.425360,1.701698e-01,5.314710e-02,-0.358775,0.954776,-0.050339,3.271456,0.889393,-40.349945
9,-23.001272,0.148865,-0.032762,-6.431885,0.233725,2.274793e-01,-2.352929e-02,-0.553863,0.045562,0.792655,0.933587,0.720231,-74.388527


In [71]:
#converting predictions to dataframe
import math
from sklearn.metrics import mean_squared_error
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', math.sqrt(mean_squared_error(pred[i], valid[i])))

rmse value for CO(GT) is :  1.4086888836870795
rmse value for PT08.S1(CO) is :  205.89558284021345
rmse value for NMHC(GT) is :  6.673548711302771
rmse value for C6H6(GT) is :  7.1300872487066185
rmse value for PT08.S2(NMHC) is :  277.8484437680451
rmse value for NOx(GT) is :  214.78322340917606
rmse value for PT08.S3(NOx) is :  244.9576966193135
rmse value for NO2(GT) is :  66.69695211711796
rmse value for PT08.S4(NO2) is :  490.08388934153083
rmse value for PT08.S5(O3) is :  446.51541648879663
rmse value for T is :  10.721325795625106
rmse value for RH is :  17.111676248172152
rmse value for AH is :  0.5216247245194149


In [88]:
pred

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,0.887831,841.93004,271.627237,1.982425,595.810006,136.652961,1119.78584,87.88094,829.685784,545.724682,10.547598,34.908644,0.437327
1,0.991857,866.300135,269.29672,2.327046,619.651929,157.767278,1098.690737,90.574363,856.150818,598.061035,9.826373,37.449595,0.442717
2,1.104181,890.877509,267.701932,2.823187,645.30167,177.409556,1078.877607,93.129588,885.360153,648.593798,9.214091,39.678858,0.447482
3,1.219169,914.856579,266.614149,3.396825,671.288989,195.619763,1060.43405,95.567038,915.323932,696.956399,8.693913,41.640506,0.45183
4,1.332886,937.732765,265.865337,3.999277,696.669347,212.431288,1043.389748,97.891452,944.774167,742.857863,8.252727,43.370069,0.455894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,2.133824,1106.975082,269.410698,10.807018,966.649034,228.624312,843.133919,101.410783,1535.018558,1040.055198,20.184181,48.901398,1.111318
1868,2.133824,1106.97508,269.410698,10.807018,966.649033,228.624308,843.133919,101.410783,1535.018563,1040.055193,20.184181,48.901398,1.111318
1869,2.133824,1106.975078,269.410699,10.807018,966.649033,228.624305,843.133918,101.410782,1535.018568,1040.055188,20.184181,48.901398,1.111318
1870,2.133824,1106.975076,269.410699,10.807018,966.649032,228.624302,843.133918,101.410781,1535.018573,1040.055184,20.184182,48.901398,1.111318


In [89]:
valid 

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date_Time,,,,,,,,,,,,,
2005-01-16 15:00:00,0.7,833,275,2.0,584,107,1144,80,821,463,11.3,32.5,0.4334
2005-01-16 16:00:00,1.1,877,275,2.8,642,176,1037,112,859,565,11.0,33.0,0.4331
2005-01-16 17:00:00,1.1,892,275,3.3,668,180,1017,121,872,632,10.3,35.0,0.4377
2005-01-16 18:00:00,1.2,899,275,3.4,674,212,1002,132,893,691,8.4,40.9,0.4542
2005-01-16 19:00:00,2.0,1008,275,7.1,861,331,839,160,977,943,8.3,38.5,0.4228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-04-04 10:00:00,3.1,1314,275,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
2005-04-04 11:00:00,2.4,1163,275,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
2005-04-04 12:00:00,2.4,1142,275,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406


In [86]:
#make final predictions
import numpy as np 
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=10)
print( np.round(yhat,2))

[[2.35000e+00 1.08633e+03 2.80760e+02 1.24100e+01 1.05536e+03 2.80880e+02
  6.59530e+02 1.68440e+02 1.15918e+03 8.50850e+02 2.73600e+01 1.55300e+01
  5.20000e-01]
 [2.45000e+00 1.09702e+03 2.84860e+02 1.26800e+01 1.05798e+03 2.93270e+02
  6.66190e+02 1.68520e+02 1.18047e+03 8.80230e+02 2.63100e+01 1.78000e+01
  5.30000e-01]
 [2.53000e+00 1.10420e+03 2.87640e+02 1.27800e+01 1.05651e+03 3.02650e+02
  6.73680e+02 1.68230e+02 1.19529e+03 9.04730e+02 2.53200e+01 1.99100e+01
  5.40000e-01]
 [2.58000e+00 1.10874e+03 2.89390e+02 1.27500e+01 1.05219e+03 3.09530e+02
  6.81730e+02 1.67600e+02 1.20540e+03 9.24980e+02 2.44100e+01 2.18800e+01
  5.50000e-01]
 [2.61000e+00 1.11134e+03 2.90340e+02 1.26500e+01 1.04594e+03 3.14340e+02
  6.90130e+02 1.66670e+02 1.21214e+03 9.41570e+02 2.35500e+01 2.37300e+01
  5.60000e-01]
 [2.62000e+00 1.11251e+03 2.90690e+02 1.24900e+01 1.03845e+03 3.17480e+02
  6.98700e+02 1.65500e+02 1.21647e+03 9.55070e+02 2.27600e+01 2.54600e+01
  5.70000e-01]
 [2.63000e+00 1.11267e

d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
d:\Users\admin\anaconda3\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [87]:
pd.DataFrame( np.round(yhat), columns=train.columns) 

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2.0,1086.0,281.0,12.0,1055.0,281.0,660.0,168.0,1159.0,851.0,27.0,16.0,1.0
1,2.0,1097.0,285.0,13.0,1058.0,293.0,666.0,169.0,1180.0,880.0,26.0,18.0,1.0
2,3.0,1104.0,288.0,13.0,1057.0,303.0,674.0,168.0,1195.0,905.0,25.0,20.0,1.0
3,3.0,1109.0,289.0,13.0,1052.0,310.0,682.0,168.0,1205.0,925.0,24.0,22.0,1.0
4,3.0,1111.0,290.0,13.0,1046.0,314.0,690.0,167.0,1212.0,942.0,24.0,24.0,1.0
5,3.0,1113.0,291.0,12.0,1038.0,317.0,699.0,165.0,1216.0,955.0,23.0,25.0,1.0
6,3.0,1113.0,291.0,12.0,1030.0,319.0,707.0,164.0,1219.0,966.0,22.0,27.0,1.0
7,3.0,1112.0,290.0,12.0,1022.0,320.0,716.0,163.0,1221.0,975.0,21.0,29.0,1.0
8,3.0,1111.0,289.0,12.0,1013.0,320.0,724.0,161.0,1221.0,982.0,21.0,30.0,1.0
9,3.0,1110.0,289.0,12.0,1005.0,319.0,732.0,159.0,1222.0,988.0,20.0,31.0,1.0


In [85]:
df

,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10 18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,2004-03-10 19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,2004-03-10 20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,2004-03-10 21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,2004-03-10 22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,2005-04-04 10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,2005-04-04 11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,2005-04-04 12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,2005-04-04 13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139
